In [ ]:
import numpy as np
from threeML import FermiGBMBurstCatalog, download_GBM_trigger_data, TimeSeriesBuilder, update_logging_level

update_logging_level("FATAL")

In [ ]:
grb_name = "GRB160509374"
data_path = "real_data/" + grb_name

In [ ]:
gbm_catalog = FermiGBMBurstCatalog()
gbm_catalog.query_sources(grb_name)

In [ ]:
det_info = gbm_catalog.get_detector_information()[grb_name]
det_info

In [ ]:
dload = download_GBM_trigger_data("bn160509374", det_info["detectors"], "/tmp/GRB")

In [ ]:
dload.items()

In [ ]:
fluence_plugins = []
time_series = {}

for i,det in enumerate(det_info["detectors"]):
    ts = TimeSeriesBuilder.from_gbm_cspec_or_ctime(
        det, cspec_or_ctime_file=dload[det]["cspec"], rsp_file=dload[det]["rsp"]
    )

    ts.set_background_interval(
        *det_info["background"]["full"].split(",")
    )
    ts.save_background(f"{data_path}/{det}_bkg.h5", overwrite=True)

    ts_tte = TimeSeriesBuilder.from_gbm_tte(
        det,
        tte_file=dload[det]["tte"],
        rsp_file=dload[det]["rsp"],
        restore_background=f"{data_path}/{det}_bkg.h5",
    )

    time_series[det] = ts_tte

    ts_tte.set_active_time_interval("9-30")

    fluence_plugin = ts_tte.to_spectrumlike()

    if det.startswith("b"):

        fluence_plugin.set_active_measurements("250-10000")

    else:

        fluence_plugin.set_active_measurements("10-900")

    fluence_plugin.rebin_on_background(1.0)

    fluence_plugins.append(fluence_plugin)

In [ ]:
%matplotlib widget
for i in time_series.values():
    i.view_lightcurve(-40,100)

In [ ]:
data = dict(
    
)